In [1]:
%cd ../../

/home/PJWSTK/s14028/engineer/Practical


In [2]:
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from augmentation import augmentation_data
from augmentation import augment_data

from CNN.CNN_v4 import CNN_v4

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn_v4 = CNN_v4((480, 640, 4), split_into_parts=20)
cnn_v4.def_model()

In [5]:
images = cnn_v4._prepare_images(image_train)
anwsers = cnn_v4._prepare_anwsers(person_coo_train)

In [6]:
ones_count = (np.sum(anwsers == 1) * 0.25).astype(np.uint32)
zeros_count = (ones_count / 0.25 * 0.75).astype(np.uint32)
validation_length = (zeros_count + ones_count).astype(np.int32)

val_indices = np.concatenate([np.where(anwsers == 0)[0][:zeros_count],
                              np.where(anwsers == 1)[0][:ones_count]])

In [7]:
anwsers[val_indices[zeros_count:]] = 0

In [8]:
val_indices = -(images.shape[0] - val_indices)

In [9]:
generator = keras.preprocessing.image.ImageDataGenerator(rotation_range=20)

augmentation = augmentation_data(image_train, anwsers, 20)
augmented_data = augment_data(generator, augmentation, images, anwsers)
images, anwsers = augmented_data

In [10]:
anwsers[val_indices[zeros_count:]] = 1

images[-validation_length:], images[val_indices] = images[val_indices], images[-validation_length:]
anwsers[-validation_length:], anwsers[val_indices] = anwsers[val_indices], anwsers[-validation_length:]

In [11]:
parallel_model = keras.utils.multi_gpu_model(
        cnn_v4.model,
        gpus=2,
        cpu_merge=False)
parallel_model.compile(
        loss="binary_crossentropy", 
        optimizer="adam", 
        metrics=["accuracy"])

In [12]:
path = "CNN/CNN_v4/weights/cnn_v4_1e_6_{}"

class ModelSaver(keras.callbacks.Callback):
    __path: str
    __current_epoch: int
        
    def __init__(self, path: str):
        self.__path = path
        self.__current_epoch = 0
        
    def on_epoch_end(self, *args, **kwargs):
        epoch_path = self.__path.format(self.__current_epoch)
        self.__current_epoch += 1
        
        parallel_model.save_weights(epoch_path)

model_saver = ModelSaver(path)

In [13]:
parallel_model.optimizer.lr.assign(1e-4)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [14]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 125s 101us/step - loss: 53.1809 - accuracy: 0.5336 - val_loss: 24.6635 - val_accuracy: 0.7500
Epoch 2/5
1244460/1244460 [==============================] - 114s 92us/step - loss: 29.5678 - accuracy: 0.5682 - val_loss: 19.4019 - val_accuracy: 0.7048
Epoch 3/5
1244460/1244460 [==============================] - 114s 92us/step - loss: 21.1116 - accuracy: 0.5838 - val_loss: 15.7836 - val_accuracy: 0.6667
Epoch 4/5
1244460/1244460 [==============================] - 114s 91us/step - loss: 16.3614 - accuracy: 0.5892 - val_loss: 12.7596 - val_accuracy: 0.6295
Epoch 5/5
1244460/1244460 [==============================] - 113s 91us/step - loss: 13.1491 - accuracy: 0.5957 - val_loss: 10.4108 - val_accuracy: 0.6252


In [15]:
parallel_model.optimizer.lr.assign(1e-4)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [16]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 113s 91us/step - loss: 10.7435 - accuracy: 0.6053 - val_loss: 8.5602 - val_accuracy: 0.6146
Epoch 2/5
1244460/1244460 [==============================] - 113s 91us/step - loss: 8.8432 - accuracy: 0.6141 - val_loss: 7.0612 - val_accuracy: 0.6173
Epoch 3/5
1244460/1244460 [==============================] - 113s 91us/step - loss: 7.3123 - accuracy: 0.6240 - val_loss: 5.8548 - val_accuracy: 0.6185
Epoch 4/5
1244460/1244460 [==============================] - 113s 91us/step - loss: 6.0698 - accuracy: 0.6303 - val_loss: 4.8676 - val_accuracy: 0.6138
Epoch 5/5
1244460/1244460 [==============================] - 114s 91us/step - loss: 5.0462 - accuracy: 0.6345 - val_loss: 4.0428 - val_accuracy: 0.6012


In [17]:
parallel_model.optimizer.lr.assign(1e-6)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-06>

In [18]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 113s 90us/step - loss: 4.5839 - accuracy: 0.6370 - val_loss: 4.2508 - val_accuracy: 0.5602
Epoch 2/5
1244460/1244460 [==============================] - 113s 91us/step - loss: 4.5753 - accuracy: 0.6364 - val_loss: 4.2607 - val_accuracy: 0.5561
Epoch 3/5
1244460/1244460 [==============================] - 113s 91us/step - loss: 4.5660 - accuracy: 0.6368 - val_loss: 4.2543 - val_accuracy: 0.5553
Epoch 4/5
1244460/1244460 [==============================] - 112s 90us/step - loss: 4.5564 - accuracy: 0.6372 - val_loss: 4.2463 - val_accuracy: 0.5552
Epoch 5/5
1244460/1244460 [==============================] - 112s 90us/step - loss: 4.5465 - accuracy: 0.6367 - val_loss: 4.2345 - val_accuracy: 0.5552


In [19]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 112s 90us/step - loss: 4.5354 - accuracy: 0.6373 - val_loss: 4.2262 - val_accuracy: 0.5554
Epoch 2/5
1244460/1244460 [==============================] - 113s 90us/step - loss: 4.5245 - accuracy: 0.6370 - val_loss: 4.2139 - val_accuracy: 0.5558
Epoch 3/5
1244460/1244460 [==============================] - 112s 90us/step - loss: 4.5133 - accuracy: 0.6371 - val_loss: 4.2028 - val_accuracy: 0.5553
Epoch 4/5
1244460/1244460 [==============================] - 112s 90us/step - loss: 4.5013 - accuracy: 0.6374 - val_loss: 4.1917 - val_accuracy: 0.5547
Epoch 5/5
1244460/1244460 [==============================] - 112s 90us/step - loss: 4.4892 - accuracy: 0.6371 - val_loss: 4.1802 - val_accuracy: 0.5555


In [20]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 113s 90us/step - loss: 4.4769 - accuracy: 0.6372 - val_loss: 4.1675 - val_accuracy: 0.5547
Epoch 2/5
1244460/1244460 [==============================] - 112s 90us/step - loss: 4.4642 - accuracy: 0.6367 - val_loss: 4.1549 - val_accuracy: 0.5548
Epoch 3/5
1244460/1244460 [==============================] - 113s 90us/step - loss: 4.4509 - accuracy: 0.6369 - val_loss: 4.1443 - val_accuracy: 0.5549
Epoch 4/5
1244460/1244460 [==============================] - 113s 90us/step - loss: 4.4373 - accuracy: 0.6368 - val_loss: 4.1352 - val_accuracy: 0.5544
Epoch 5/5
1244460/1244460 [==============================] - 112s 90us/step - loss: 4.4235 - accuracy: 0.6367 - val_loss: 4.1189 - val_accuracy: 0.5542


In [21]:
parallel_model.optimizer.lr.assign(1e-4)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [22]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 112s 90us/step - loss: 3.8071 - accuracy: 0.6388 - val_loss: 2.7945 - val_accuracy: 0.5905
Epoch 2/5
1244460/1244460 [==============================] - 112s 90us/step - loss: 2.8608 - accuracy: 0.6425 - val_loss: 2.1674 - val_accuracy: 0.5828
Epoch 3/5
1244460/1244460 [==============================] - 111s 89us/step - loss: 2.2517 - accuracy: 0.6431 - val_loss: 1.7844 - val_accuracy: 0.5716
Epoch 4/5
1244460/1244460 [==============================] - 111s 90us/step - loss: 1.8437 - accuracy: 0.6431 - val_loss: 1.5242 - val_accuracy: 0.5584
Epoch 5/5
1244460/1244460 [==============================] - 111s 90us/step - loss: 1.5787 - accuracy: 0.6458 - val_loss: 1.3670 - val_accuracy: 0.5362
